In [1]:
import pandas as pd
import numpy as np
import keras
import glob
import os
import random
import pickle
from skimage import io
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, InputLayer
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import ZeroPadding2D, Dropout
from tensorflow.compat.v1.keras.preprocessing.image import ImageDataGenerator

In [2]:
with open('X_train', 'rb') as f:
    X_train = pickle.load(f)
with open('X_test', 'rb') as f:
    X_test = pickle.load(f)
with open('y_train', 'rb') as f:
    y_train = pickle.load(f)
with open('y_test', 'rb') as f:
    y_test = pickle.load(f)

In [3]:
# base = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))
base = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(128,128,3))
# base = NASNetLarge(weights='imagenet', include_top=False, input_shape=(128,128,3))
# base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
# base = Xception(weights='imagenet', include_top=False, input_shape=(128,128,3))

In [4]:
for layer in base.layers:
    layer.trainable = False  

In [5]:
# datagen = ImageDataGenerator(
#     rotation_range = 355,
#     vertical_flip = True,
#     horizontal_flip = True,
#     validation_split = 0.25)

In [6]:
# datagen.fit(X_train)

In [7]:
# DCN structure

# **************
perceptrons = 32
# activationFunction = 'relu'
# activationFunction = 'LeakyReLU'
activationFunction = 'elu'

DCN = Sequential(
                [InputLayer(input_shape=X_train.shape[1:]), # 128x128x3 input layer
                 base,
                 Flatten(),
                 # **************
                 Dense(perceptrons, activation=activationFunction), # Hidden 1
                 Dense(perceptrons, activation=activationFunction), # Hidden 2
                 Dense(perceptrons, activation=activationFunction), # Hidden 3
                 Dense(perceptrons, activation=activationFunction), # Hidden 4
#                *****************************
                 Dense(perceptrons, activation=activationFunction), # Hidden 5
                 Dense(perceptrons, activation=activationFunction), # Hidden 6
                 Dense(perceptrons, activation=activationFunction), # Hidden 7
                 Dense(perceptrons, activation=activationFunction), # Hidden 8
                 Dense(4, activation='softmax')]
               )

DCN.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), # **************
    metrics=['accuracy'],
)

In [14]:
DCN.fit(X_train, y_train, batch_size=64, epochs=8, verbose=1, validation_split=0.25) # **************

Epoch 1/8
40/40 [==============================] - 161s 4s/step - loss: 0.1563 - accuracy: 0.9427 - val_loss: 0.3015 - val_accuracy: 0.8902
Epoch 2/8
40/40 [==============================] - 163s 4s/step - loss: 0.1318 - accuracy: 0.9542 - val_loss: 0.4294 - val_accuracy: 0.8878
Epoch 3/8
40/40 [==============================] - 165s 4s/step - loss: 0.1205 - accuracy: 0.9562 - val_loss: 0.3384 - val_accuracy: 0.9010
Epoch 4/8
40/40 [==============================] - 164s 4s/step - loss: 0.1277 - accuracy: 0.9582 - val_loss: 0.3386 - val_accuracy: 0.8914
Epoch 5/8
40/40 [==============================] - 163s 4s/step - loss: 0.1002 - accuracy: 0.9618 - val_loss: 0.2947 - val_accuracy: 0.9033
Epoch 6/8
40/40 [==============================] - 165s 4s/step - loss: 0.1328 - accuracy: 0.9622 - val_loss: 0.2615 - val_accuracy: 0.9081
Epoch 7/8
40/40 [==============================] - 167s 4s/step - loss: 0.3387 - accuracy: 0.8985 - val_loss: 0.5225 - val_accuracy: 0.8473
Epoch 8/8
40/40 [===

In [9]:
# DCN.fit(datagen.flow(X_train, y_train, batch_size=64,
#          subset='training'),
#          validation_data=datagen.flow(X_train, y_train,
#          batch_size=64, subset='validation'),
#          batch_size=64, epochs=10)

In [10]:
y_train_cat = []
for y in y_test:
    for k in range(len(y)):
        if y[k] == 1:
            y_train_cat.append(k)
            break

In [11]:
accuracy_score(y_train_cat, np.argmax(DCN.predict(X_test), axis=-1))

27/27 [==============================] - 47s 2s/step


0.8949880668257757

In [12]:
# y_train_cat

In [13]:
# Priorities

# look into if we can replicate a different NN-Structure from videos
# Create non-deep learning approach (no activation function, no hidden layers, just linear regression)
# complete EDA
# data augmentation with keras data generation
# possibly other activation functions?